In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/updatetraintest/test_set.csv
/kaggle/input/updatetraintest/train_set.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW,AutoTokenizer

In [5]:
from sklearn.model_selection import KFold

In [6]:
df=pd.read_csv("/kaggle/input/updatetraintest/questionequation.csv")

In [7]:

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, text_list, equation_list, tokenizer, max_length=256):
        self.text_list = text_list
        self.equation_list = equation_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = str(self.text_list[idx])
        equation = str(self.equation_list[idx])

        inputs = self.tokenizer(
            f"text: {text}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        labels = self.tokenizer(
            f"equation: {equation}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze()
        }


In [8]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model directly

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")



tokenizer = AutoTokenizer.from_pretrained("t5-base",model_max_length=256)

# Set device to CPU
device = torch.device("cuda")
model.to(device)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [9]:
training_loss=[]
validation_loss=[]

In [10]:
dataloader = DataLoader(dataset=df, batch_size=2, shuffle=True)

In [12]:
df= df.iloc[:5001]

In [ ]:
num_epochs = 2  # Adjust as needed
k_folds = 5
kf = KFold(n_splits=k_folds)

fold_losses = []

for fold, (train_index, val_index) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Create train and validation datasets
    train_data = df.iloc[train_index]
    val_data = df.iloc[val_index]

    train_dataset = CustomDataset(
        text_list=train_data.iloc[:,0].tolist(),
        equation_list=train_data.iloc[:,1].tolist(),
        tokenizer=tokenizer
    )

    val_dataset = CustomDataset(
        text_list=val_data.iloc[:,0].tolist(),
        equation_list=val_data.iloc[:,1].tolist(),
        tokenizer=tokenizer
    )

    # Create DataLoader for training and validation
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=2, shuffle=False)

    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=5e-5)

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0.0
        model.train()
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        training_loss.append(average_loss)
        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {average_loss:.4f}")

    #Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_dataloader)
        validation_loss.append(average_val_loss)
        print(f"Validation Loss: {average_val_loss:.4f}")
        fold_losses.append(average_val_loss)
    

In [ ]:
training_loss


In [ ]:
validation_loss

In [ ]:
model.save_pretrained("/kaggle/working/epochs")


In [16]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

# **Accuracy Calculation: Direct comparision**

In [ ]:
acct=0
inputs=list(X_test)
import re
outputs=list(Y_test)
for input,real in zip(inputs,outputs):
       encoded_input = tokenizer(input, padding=True, truncation=True, return_tensors='pt')
       input_ids, attention_mask = encoded_input.input_ids, encoded_input.attention_mask
       input_ids = input_ids.to(device)
       attention_mask = attention_mask.to(device)
       output = model.generate(input_ids,max_length=200)
       pred = tokenizer.decode(output[0], skip_special_tokens=False)
       pred = pred.replace('<pad>','')
       pred = pred.replace('<unk>','-')
       pred = pred.replace('</s>','')
       pred = pred.replace("equation:", '')
       answer1= solve_equation(pred)
       answer2 = solve_equation(real)
       if(answer1 == answer2 ):
           acct=acct+1
result=(acct/len(inputs))*100


# Accuracy Calculation:Final answer comparision

In [ ]:
from fractions import Fraction

In [ ]:
import re
import sympy as sp

def standardize_equation_onevar(equation):

    equation = equation.replace(" ", "").lower()

    # Use regular expressions to find the variable name (any alphabet character)
    variable_match = re.search(r'[a-z]', equation)

    if variable_match:
        variable = variable_match.group()
    else:
        raise ValueError("No variable found in the equation.")


    terms = re.split(r'([-+=])', equation)
    standardized_equation = ""

    for i, term in enumerate(terms):
        if term == variable and (i == 0 or terms[i - 1] in ['=', '+', '-']):

            standardized_equation += term
        elif variable in term:

            standardized_equation += term.replace(variable, '*' + variable)
        else:

            standardized_equation += term

    return variable

def standardize_equation_twovar(equation):

    equation = equation.replace(" ", "").lower()


    variables = set(re.findall(r'[a-z]', equation))

    if len(variables) == 0:
        raise ValueError("No variables found in the equation.")


    terms = re.split(r'([-+=])', equation)
    standardized_equation = ""

    for i, term in enumerate(terms):

        if term in variables and (i == 0 or terms[i - 1] in ['=', '+', '-']):

            standardized_equation += term
        elif any(var in term for var in variables):

            for var in variables:
                term = term.replace(var, '*' + var)
            standardized_equation += term
        else:

            standardized_equation += term

    return variables


def solve_onevar(equation):
  try:
    eqn_var = standardize_equation_onevar(equation)
    equations1 = equation.split(',')
    x= sp.symbols(eqn_var)
    for eq_str in equations1:
 
        left_side, right_side = eq_str.split('=')
    left_expr = sp.sympify(left_side)
    right_expr = sp.sympify(right_side)
    equation = sp.Eq(left_expr, right_expr)
    solution = sp.solve(equation, x)
    return solution
  except:
    return [0]

def solve_twovar(equation):
  try:
    total_eqns = equation.split(',')
    eqn_one = total_eqns[0]
    eqn_two = total_eqns[1]
    symbols1 = standardize_equation_twovar(eqn_one)
    symbols2 = standardize_equation_twovar(eqn_two)
    if(len(symbols1)>len(symbols2)):
      symbols=symbols1
    else:
      symbols=symbols2
    x,y= sp.symbols(symbols)
    left_side, right_side = eqn_one.split('=')
    right_expr = sp.sympify(right_side)
    left_expr = sp.sympify(left_side)
    finaleqn_1 = sp.Eq(left_expr, right_expr)
    left_side, right_side = eqn_two.split('=')
    right_expr = sp.sympify(right_side)
    left_expr = sp.sympify(left_side)
    finaleqn_2 = sp.Eq(left_expr, right_expr)
    solutions = sp.solve((finaleqn_1, finaleqn_2), (x, y))
    if isinstance(solutions, dict):
       my_list = [value for value in solutions.values()]
   # print(solutions)
       return my_list
    elif isinstance(solutions, list):
      if(len(solutions)==2):
         return solutions
      else:
        return [0,0]
    else:
      return [0,0]
  except:
    return [0,0]



In [ ]:
acct = 0
model.eval()

inputs = list(X_test)
outputs = list(y_test)
for input, real in zip(inputs, outputs):
    encoded_input = tokenizer(input, padding=True, truncation=True, return_tensors='pt')
    input_ids, attention_mask = encoded_input.input_ids, encoded_input.attention_mask
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    output = model.generate(input_ids)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    eqn = pred
    ansx=0
    ansy=0
   
    total_eqns = eqn.split(',')
    if len(total_eqns) > 1:
        ans = solve_twovar(eqn)
        if(len(ans)==1):
           ansx = ans[0]
           ansy=0
        else:
          ansy = ans[1]
          ansx = ans[0]
    else:
        try:

            ans=solve_onevar(eqn)
            if(type(ans[0])==int  or float):
                 ansx = ans[0]
            else:
                ansx=0
        except:
          ansx=0



    eqn = real
    total_eqns = eqn.split(',')
    if len(total_eqns) > 1:
        ansr = solve_twovar(eqn)
        if(len(ansr)==1):
           ansrx = ansr[0]
           ansry = 0
        else:
          ansrx = ansr[0]
          ansry = ansr[1]
        if( ansx == ansrx and ansy == ansry) or (ansx==ansry and ansy==ansrx) :
            acct = acct + 1
    else:
        ansr = solve_onevar(eqn)
        if(len(ansr)==1):

          ansrx = ansr[0]
        else:
          ansrx=0
        print(ansrx)
        if ansx == ansrx:
            acct = acct + 1
result = (acct / len(inputs)) * 100


# Using the Rouge Score

In [ ]:
pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
rouge1sump=0
rouge1sumr=0
rouge1sumf=0
rouge2sump=0
rouge2sumr=0
rouge2sumf=0
rougelsump=0
rougelsumr=0
rougelsumf=0

In [ ]:
    acct=0
    model.eval()
    inputs=list(X_test)
    outputs=list(Y_test)
    for input,real in zip(inputs,outputs):
       encoded_input = tokenizer(input, padding=True, truncation=True, return_tensors='pt')
       input_ids, attention_mask = encoded_input.input_ids, encoded_input.attention_mask
       input_ids = input_ids.to(device)
       attention_mask = attention_mask.to(device)
       output = model.generate(input_ids,max_length=200)
       pred=tokenizer.decode(output[0], skip_special_tokens=True)
       pred = str(pred).replace("equation:", "")
       pred = str(pred).replace(" ", "")
       real = str(real).replace(" ", "")
       if(pred==real):
           acct=acct+1
       scores = scorer.score(real, pred)
       for key in scores:
              if(key=="rouge1"):
                    print("rouge1")
                    rouge1sump+=scores[key].precision
                    rouge1sumr+=scores[key].recall
                    rouge1sumf+=scores[key].fmeasure
              elif(key=="rouge2"):
                    print("rouge2")
                    rouge2sump+=scores[key].precision
                    rouge2sumr+=scores[key].recall
                    rouge2sumf+=scores[key].fmeasure
              elif(key=="rougeL"):
                    rougelsump+=scores[key].precision
                    rougelsumr+=scores[key].recall
                    rougelsumf+=scores[key].fmeasure
              else:
                print("wrong")

avg1p=rouge1sump/len(inputs)
avg1r=rouge1sumr/len(inputs)
avg1f=rouge1sumf/len(inputs)
avg2p=rouge2sump/len(inputs)
avg2r=rouge2sumr/len(inputs)
avg2f=rouge2sumf/len(inputs)
avglp=rougelsump/len(inputs)
avglr=rougelsumr/len(inputs)
avglf=rougelsumf/len(inputs)


result=(acct/len(inputs))*100
